In [12]:
import logging
from typing import List

import pandas as pd
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.evaluation import EmbeddingDistance
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain.evaluation.schema import StringEvaluator
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import QAGenerationChain
from langchain import PromptTemplate

In [5]:
loader = TextLoader("../state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [3]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [6]:
hf_embeddings_vector_db = FAISS.from_documents(docs, hf_embeddings)
hf_embeddings_vector_db.save_local("faiss_index")

In [7]:
question = "What did the president say about economics?"
docs = hf_embeddings_vector_db.similarity_search(question)
docs[0].page_content

'We’re going after the criminals who stole billions in relief money meant for small businesses and millions of Americans.  \n\nAnd tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. \n\nBy the end of this year, the deficit will be down to less than half what it was before I took office.  \n\nThe only president ever to cut the deficit by more than one trillion dollars in a single year. \n\nLowering your costs also means demanding more competition. \n\nI’m a capitalist, but capitalism without competition isn’t capitalism. \n\nIt’s exploitation—and it drives up prices. \n\nWhen corporations don’t have to compete, their profits go up, your prices go up, and small businesses and family farmers and ranchers go under. \n\nWe see it happening with ocean carriers moving goods in and out of America. \n\nDuring the pandemic, these foreign-owned companies raised prices by as much as 1,000% and made record profits.'

In [8]:
gpt35_azure_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_key="02e3dbabaf334ccb959cbeadbd3f99c3",
    openai_api_base="https://llm-x-gpt.openai.azure.com/",
    deployment_name='LLM-X-GPT35-TURBO',
    openai_api_version="2023-03-15-preview"
)

In [9]:
gpt4_azure_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_key="a8d69f68a36b40789df2cc3fdbaacda9",
    openai_api_base="https://llmx-gpt-canada-east.openai.azure.com/",
    deployment_name='LLM-X-GPT-4',
    openai_api_version="2023-03-15-preview"
)

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings

openai_azure_embeddings = OpenAIEmbeddings(
    openai_api_key="02e3dbabaf334ccb959cbeadbd3f99c3",
    openai_api_base="https://llm-x-gpt.openai.azure.com/",
    deployment_name='LLM-X-Embedding'
)

/home/watson/.virtualenvs/QnAEvaluation/lib/python3.9/site-packages/langchain/embeddings/openai.py:214: UserWarning: WARNING! deployment_name is not default parameter.
                    deployment_name was transferred to model_kwargs.
                    Please confirm that deployment_name is what you intended.
  warnings.warn(


In [11]:
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=hf_embeddings_vector_db.as_retriever(),
    llm=gpt35_azure_llm
)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ["1. Can you provide any information on the president's statements regarding the field of economics?", "2. I'm interested in knowing the president's views and comments on the subject of economics. Could you share any relevant information?", '3. Could you please share any insights or remarks made by the president in relation to economics?']


In [38]:
from langchain.chains import RetrievalQA

qa_rag_chain = RetrievalQA.from_chain_type(
    llm=gpt35_azure_llm,
    retriever=hf_embeddings_vector_db.as_retriever()
)

question = "What did the president say about Zelensky?"
qa_rag_chain({"query": question})

{'query': 'What did the president say about Zelensky?',
 'result': 'The president mentioned President Zelenskyy of Ukraine and praised the fearlessness, courage, and determination of the Ukrainian people.'}

In [13]:
templ = """You are a smart assistant designed to help high school teachers come up with reading comprehension questions.
Given a piece of text, you must come up with a {k} different question and answer pairs that can be used to test a student's reading comprehension abilities.
When coming up with this question/answer pair, each pair must be respond in the following format:

{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}}

So in your final answer you should response with a list of {k} pairs in this format:

```
[{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}},
 {{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}},
 {{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
    }}
]
```

Please come up with a list of {k} question/answer pairs, in the specified list of JSONS format, for the following text:
----------------
{text}
"""

multi_qa_prompt_gpt35 = PromptTemplate.from_template(template=templ, partial_variables={"k": 5})
qa_generation_chain_gpt35 = QAGenerationChain.from_llm(llm=gpt35_azure_llm, prompt=multi_qa_prompt_gpt35)

In [14]:
multi_qa_prompt_template = """You are a smart assistant designed to help high school teachers come up with reading comprehension questions.
Given a piece of text, you must come up with a {k} different question and answer pairs that can be used to test a student's reading comprehension abilities.
When coming up with this question/answer pair, each pair must be respond in the following format:

{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}}
Please come up with a list of {k} question/answer pairs, in the specified list of dict, for the following text:
----------------
{text}
"""

In [14]:
def create_qna_GT_df(docs: List[Document], num_of_qna_for_doc: int) -> pd.DataFrame:
    multi_qa_prompt = PromptTemplate.from_template(template=multi_qa_prompt_template,
                                                   partial_variables={"k": num_of_qna_for_doc})
    qa_generation_chain = QAGenerationChain.from_llm(llm=gpt35_azure_llm, prompt=multi_qa_prompt_gpt35)

    qna_GT = []
    for doc in docs:
        doc_qna = qa_generation_chain.run(doc.page_content)[0]
        qna_GT += doc_qna

    qna_GT_df = pd.DataFrame(qna_GT)
    return qna_GT_df

In [15]:
def get_qna_with_chain_answers_df(qa_rag_chain: RetrievalQA, qna_GT_df: pd.DataFrame) -> pd.DataFrame:
    qna_with_chain_answers = qna_GT_df.copy()
    qna_with_chain_answers["chain_answer"] = qna_with_chain_answers.apply(
        lambda qna: qa_rag_chain({"query": qna["question"]})["result"], axis=1)

    return qna_with_chain_answers

In [16]:
def get_evaluator_score(evaluator: StringEvaluator, qna_with_chain_answer: pd.Series) -> float:
    grade = evaluator.evaluate_strings(
        prediction=qna_with_chain_answer["chain_answer"],
        reference=qna_with_chain_answer["answer"],
        input=qna_with_chain_answer["question"])

    return grade["score"]

In [17]:
def get_grades_for_chain_qna(qna_with_chain_answers_df: pd.DataFrame) -> pd.DataFrame:
    grades_for_chain_qna = qna_with_chain_answers_df.copy()

    labeled_criteria_evaluator = load_evaluator(evaluator=EvaluatorType.LABELED_CRITERIA,
                                                criteria="correctness",
                                                llm=gpt35_azure_llm)

    embedding_distance_evaluator = load_evaluator(evaluator=EvaluatorType.EMBEDDING_DISTANCE,
                                                  distance_metric=EmbeddingDistance.COSINE,
                                                  embeddings=hf_embeddings,
                                                  llm=gpt35_azure_llm)

    qa_llm__jugde_evaluator = load_evaluator(evaluator=EvaluatorType.QA,
                                             llm=gpt35_azure_llm)

    grades_for_chain_qna["labeled_criteria_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=labeled_criteria_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    grades_for_chain_qna["embedding_distance_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=embedding_distance_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    grades_for_chain_qna["qa_llm_jugde_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=qa_llm__jugde_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    return grades_for_chain_qna

In [18]:
def run_evaluation(qa_rag_chain: RetrievalQA, qna_GT_df: pd.DataFrame) -> pd.DataFrame:
    qna_with_chain_answers_df = get_qna_with_chain_answers_df(qa_rag_chain, qna_GT_df)
    grades_for_chain_qna = get_grades_for_chain_qna(qna_with_chain_answers_df)
    return grades_for_chain_qna

In [19]:
qna_GT_df = create_qna_GT_df(docs=docs, num_of_qna_for_doc=2)
run_evaluation(qa_rag_chain=qa_rag_chain, qna_GT_df=qna_GT_df)

/home/watson/.virtualenvs/QnAEvaluation/lib/python3.9/site-packages/langchain/evaluation/schema.py:104: UserWarning: Ignoring input in EmbeddingDistanceEvalChain, as it is not expected.
  warn(self._skip_input_warning)


,question,answer,chain_answer,labeled_criteria_grades,embedding_distance_grades,qa_llm_jugde_grades
0,What is the purpose of the chief prosecutor fo...,To go after the criminals who stole relief mon...,The purpose of the chief prosecutor for pandem...,1.0,0.432731,1
1,How much will the deficit be reduced to by the...,Less than half of what it was before the curre...,The given context does not provide specific in...,1.0,0.604303,1
2,What does the speaker mean by 'capitalism with...,The speaker believes that true capitalism requ...,The speaker means that true capitalism require...,1.0,0.098474,1
3,What happens to prices when corporations don't...,Prices go up.,"When corporations don't have to compete, their...",1.0,0.469043,1
4,What did foreign-owned ocean carriers do durin...,"They raised prices by as much as 1,000% and ma...","During the pandemic, foreign-owned ocean carri...",1.0,0.242672,1
5,What is the author's plan to fight inflation?,"Lowering costs, making more cars and semicondu...",The author's plan to fight inflation includes ...,0.0,0.462797,1
6,What do economists call the author's plan?,Increasing the productive capacity of the econ...,"Economists call the author's plan ""increasing ...",1.0,0.281416,1
7,What does the author call his plan?,Building a better America.,"The author calls his plan ""building a better A...",1.0,0.380859,1
8,"According to the author, how will his plan low...",By cutting the cost of prescription drugs.,The author's plan to fight inflation and lower...,1.0,0.404212,1
9,Who supports the author's plan?,Top business leaders and most Americans.,The author mentions that 17 Nobel laureates in...,1.0,0.487066,1


In [15]:
grade_docs_prompt_template = """
    You are a grader trying to determine if a set of retrieved documents will help a student answer a question. \n

    Here is the question: \n
    {query}

    Here are the documents retrieved to answer question: \n
    {result}

    Here is the correct answer to the question: \n
    {answer}

    Criteria:
      relevance: Do all of the documents contain information that will help the student arrive that the correct answer to the question?"

    Your response should be as follows:

    GRADE: (Correct or Incorrect, depending if all of the documents retrieved meet the criterion)
    (line break)
    JUSTIFICATION: (Write out in a step by step manner your reasoning about the criterion to be sure that your conclusion is correct. Use three sentences maximum. Keep the answer as concise as possible.)
    """

GRADE_DOCS_PROMPT = PromptTemplate(input_variables=['result', 'answer', 'query'],
                                   template=grade_docs_prompt_template)

In [42]:
question = "What did the president say about capitalism?"
answer =  "The president said, 'I’m a capitalist, but capitalism without competition isn’t capitalism. It’s exploitation—and it drives up prices. When corporations don’t have to compete, their profits go up, your prices go up, and small businesses and family farmers and ranchers go under."

In [55]:
hf_retriever = hf_embeddings_vector_db.as_retriever()
retrieved_docs = hf_retriever.get_relevant_documents(query=question,
                                                     search_kwargs={"k": 2})

l = qa_rag_chain.retriever.get_relevant_documents(query=question,
                                                  search_type="similarity_score_threshold",
                                                  search_kwargs={"k": 2})

In [56]:
eval_chain = load_evaluator(
    evaluator=EvaluatorType.QA,
    llm=gpt35_azure_llm,
    prompt=GRADE_DOCS_PROMPT
)

In [57]:
graded_outputs = eval_chain.evaluate_strings(
        prediction=l[0].page_content,
        reference=answer,
        input=question
)

In [58]:
graded_outputs

{'reasoning': 'GRADE: Correct',
 'value': 'JUSTIFICATION: All of the documents retrieved contain information that directly addresses the question about what the president said about capitalism. The documents include statements from the president explicitly discussing capitalism, competition, exploitation, and the impact on prices, small businesses, and farmers. Therefore, all of the documents are relevant and will help the student arrive at the correct answer.',
 'score': 1}